## Webscarpping and structuring results

In [1]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
def get_page_content(url):
    loader = WebBaseLoader(url)
    page_data = loader.load().pop().page_content
    return page_data

In [3]:
get_page_content("https://www.infineon.com/cms/en/careers/jobsearch/jobsearch/#!view=jobs&functional_area=Data%20Science&country=Germany&job_attributes=Full%20time")

'\n\n\n\n  \n\n\n\n\nJob Search - Infineon Technologies\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nToggle Navigation\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\nProducts\n\n\nApplications\n\n\nDesign Support\n\n\nCommunity\n\n\nAbout Infineon\n\n\nCareers\n\n\n\n\n\n\n\nNewsletter\nContact\nWhere to Buy\n\nEnglish\n\n\n简体中文\nDeutsch\n日本語\n\n\n\nmyInfineon\n\n\n\n\nRegister for myInfineon\n\n\n\n\nCart\n\n\n\n\n\n\n\n\n\n\n\nCompany\nOur Divisions\nManagement Board\nSupervisory Board\nOur Locations\nProcurement\nQuality\nInfineon Awards\nInfineon Academy I Training\nCybersecurity\n\n\nSustainability\nEnvironmental Sustainability and Climate Protection\nSafety and Health\nBusiness Ethics\nCorporate Citizenship\nCSR Supply Chain Management\nHuman Rights\nCSR Reporting\n\n\n\n\nPress\nGeneral Information\nPress Releases\nMarket News\nPress Kits\nMedia Pool\nEvents\nContacts\n\n\nInvestor\nAnnual Gene

#### TO DO: Aim is to convert above web scraped content into jason, so that it is usable

##### I will pass webscrapped content along with a prompt asking an LLM to convert given text into dict format

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
) 

In [5]:
## Getting an LLM Model [llama-3.1], (use of groq cloud for faster inference)

from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    groq_api_key = "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",
    temperature=0,
    # max_tokens=None,
    # timeout=None,
    # max_retries=2,
    # # other params...
)

In [6]:
chain_extract = prompt_extract | llm  ## forming chain using pipe operation

In [12]:
url = "https://jobs.bosch.com/en/job/REF221607P-phd-predictive-maintenance-in-manufacturing"
page_data = get_page_content(url)
page_data 

'\n      PhD Predictive Maintenance in Manufacturing                                                                  Skip to main content                                          Close                Bosch Career Portal         PhD Predictive Maintenance in Manufacturing              Location Salzgitter        Fields of work  Manufacturing         Join as  Graduate         Starting date According to arrangement        Working time Vollzeit        Legal entity Robert Bosch Elektronik GmbH            Bosch data privacy statement eRecruiting     SmartRecruiters data privacy statement          Your tasks Als PhD Kandidat bist du Schnittstelle zwischen Forschung und Industrie. Du erforschst die Integration von Predictive Maintenance in das Bosch Production System.\xa0• Du identifizierst Potentiale für Predictive Maintenance in\n\xa0 unserer Elektronikfertigung.\n• Du erforschst die Integrationsmöglichkeiten von Predictive\n\xa0 Maintenance in eine existierende\n\xa0 Wartungsprozesslandscha

In [13]:
res = chain_extract.invoke(input={'page_data':page_data})
res.content

'[\n  {\n    "role": "PhD Predictive Maintenance in Manufacturing",\n    "experience": "Master\'s degree in a relevant field, experience in manufacturing and predictive maintenance",\n    "skills": "Programming skills, knowledge of manufacturing systems, excellent communication skills, ability to work independently and in a team",\n    "description": "The PhD candidate will research the integration of predictive maintenance in the Bosch Production System, identify potential for predictive maintenance in electronics manufacturing, and develop intelligent solutions with cross-functional teams."\n  }\n]'

In [14]:
type(res.content)

str

##### note: need to convert str type to jason or dict format

In [15]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'PhD Predictive Maintenance in Manufacturing',
  'experience': "Master's degree in a relevant field, experience in manufacturing and predictive maintenance",
  'skills': 'Programming skills, knowledge of manufacturing systems, excellent communication skills, ability to work independently and in a team',
  'description': 'The PhD candidate will research the integration of predictive maintenance in the Bosch Production System, identify potential for predictive maintenance in electronics manufacturing, and develop intelligent solutions with cross-functional teams.'}]

In [16]:
type(json_res[0])

dict